<a href="https://colab.research.google.com/github/Ujwal-EVA/Assignment-8/blob/main/EVA3_Session_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Preparation with Albumentations

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import CIFAR10
import numpy as np
import torch

# CIFAR-10 mean and std
CIFAR10_MEAN = [0.4914, 0.4822, 0.4465]
CIFAR10_STD = [0.2023, 0.1994, 0.2010]

# Albumentations Transformations
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16,
                    min_holes=1, min_height=16, min_width=16,
                    fill_value=CIFAR10_MEAN, mask_fill_value=None, p=0.5),
    A.Normalize(mean=CIFAR10_MEAN, std=CIFAR10_STD),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Normalize(mean=CIFAR10_MEAN, std=CIFAR10_STD),
    ToTensorV2()
])

# Custom Dataset Class to Apply Albumentations
class CIFAR10Albumentations(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = np.array(image)  # Convert PIL image to numpy array
        if self.transform:
            augmented = self.transform(image=image)  # Apply transformations
            image = augmented['image']
        return image, label

# Load CIFAR-10 datasets
train_data = CIFAR10(root='./data', train=True, download=True)
test_data = CIFAR10(root='./data', train=False, download=True)

# Create custom datasets with albumentations
train_dataset = CIFAR10Albumentations(train_data, transform=train_transform)
test_dataset = CIFAR10Albumentations(test_data, transform=test_transform)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


100%|██████████| 170M/170M [00:03<00:00, 43.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Model Architecture

In [ ]:
import torch.nn as nn
from torchsummary import summary

class CIFAR10Model(nn.Module):
    def __init__(self):
        super(CIFAR10Model, self).__init__()

        # Input Block
        self.input_block = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )

        # Convolution Block 1: Standard Convolution
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )

        # Convolution Block 2: Depthwise Separable Convolution
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, groups=32, bias=False),
            nn.Conv2d(64, 64, kernel_size=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        # Convolution Block 3: Dilated Convolution
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=2, dilation=2, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        # Output Block: Strided Convolution
        self.output_block = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        # Global Average Pooling and Fully Connected Layer
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(128, 10)  # Reduced output channel size

    def forward(self, x):
        x = self.input_block(x)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.output_block(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# Instantiate the model and move it to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CIFAR10Model().to(device)

# Display the torch summary
print("Model Summary:")
summary(model, input_size=(3, 32, 32))


Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 32, 32, 32]           4,608
       BatchNorm2d-5           [-1, 32, 32, 32]              64
              ReLU-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 64, 32, 32]             576
            Conv2d-8           [-1, 64, 32, 32]           4,096
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
           Conv2d-11           [-1, 64, 32, 32]          36,864
      BatchNorm2d-12           [-1, 64, 32, 32]             128
             ReLU-13           [-1, 64, 32, 32]               0
           Conv2d-14    

Training and Visualization

In [ ]:
from tqdm import tqdm
import torch

# Training function with progress bar
def train_model_with_progress_bar(model, train_loader, test_loader, criterion, optimizer, epochs=10):
    model.train()
    train_loss = []
    train_accuracy = []

    for epoch in range(epochs):
        epoch_loss = 0
        correct = 0
        total = 0

        # Progress bar for training
        with tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}") as t:
            for images, labels in t:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # Calculate loss and accuracy
                epoch_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

                # Update progress bar description
                t.set_postfix(loss=epoch_loss / len(train_loader), accuracy=100. * correct / total)

        # Append epoch loss and accuracy
        train_loss.append(epoch_loss / len(train_loader))
        train_accuracy.append(100. * correct / total)
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {train_loss[-1]:.4f}, Accuracy: {train_accuracy[-1]:.2f}%")

    return train_loss, train_accuracy

# Test function to evaluate on test data
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    avg_loss = test_loss / len(test_loader)
    accuracy = 100. * correct / total
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
    return avg_loss, accuracy

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train and evaluate the model
epochs = 200
train_loss, train_accuracy = train_model_with_progress_bar(model, train_loader, test_loader, criterion, optimizer, epochs)
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

# Plot training loss and accuracy as bar graphs
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

def plot_training_results(train_loss, train_accuracy, epochs):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Loss bar graph
    ax1.bar(range(1, epochs + 1), train_loss, color='skyblue', edgecolor='black')
    ax1.set_xlabel("Epochs", fontsize=14)
    ax1.set_ylabel("Average Training Loss", fontsize=14)
    ax1.set_title("Training Loss per Epoch", fontsize=16)
    ax1.set_xticks(range(1, epochs + 1))
    ax1.grid(axis='y', linestyle='--', alpha=0.7)

    # Accuracy bar graph
    ax2.bar(range(1, epochs + 1), train_accuracy, color='limegreen', edgecolor='black')
    ax2.set_xlabel("Epochs", fontsize=14)
    ax2.set_ylabel("Training Accuracy (%)", fontsize=14)
    ax2.set_title("Training Accuracy per Epoch", fontsize=16)
    ax2.set_xticks(range(1, epochs + 1))
    ax2.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.show()

Epoch 1/200: 100%|██████████| 391/391 [00:29<00:00, 13.40it/s, accuracy=84, loss=0.459]


Epoch [1/200] Loss: 0.4587, Accuracy: 84.01%


Epoch 2/200: 100%|██████████| 391/391 [00:29<00:00, 13.25it/s, accuracy=83.8, loss=0.461]


Epoch [2/200] Loss: 0.4612, Accuracy: 83.76%


Epoch 3/200: 100%|██████████| 391/391 [00:29<00:00, 13.37it/s, accuracy=84.1, loss=0.454]


...

Epoch [128/200] Loss: 0.3918, Accuracy: 86.36%


Epoch 129/200: 100%|██████████| 391/391 [00:30<00:00, 12.91it/s, accuracy=86.2, loss=0.391]


Epoch [129/200] Loss: 0.3914, Accuracy: 86.24%


Epoch 130/200:  72%|███████▏  | 281/391 [00:22<00:08, 12.66it/s, accuracy=85.8, loss=0.288]


KeyboardInterrupt: 